In [1]:
import torch

In [2]:
from torchvision import models

In [3]:
dir(models)

['AlexNet',
 'DenseNet',
 'Inception3',
 'ResNet',
 'SqueezeNet',
 'VGG',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'inception',
 'inception_v3',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn']

In [4]:
alexnet = models.AlexNet()
alexnet

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p

In [5]:
import os
os.chdir("c:/Users/soohan/ML_repository_study/data")
resnet = models.resnet18(pretrained = True)
os.chdir("c:/Users/soohan/ML_repository")

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\soohan/.cache\torch\checkpoints\resnet18-5c106cde.pth
100%|█████████████████████████████████████████████████████████████████| 46827520/46827520 [00:06<00:00, 6970708.39it/s]


In [6]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [7]:
import torch
import torch.nn as nn
class ResNetBlock(nn.Module):
    def __init__(self, dim):
        super(ResNetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim)
    def build_conv_block(self, dim):
        conv_block = []
        conv_block += [nn.ReflectionPad2d(1)]
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True),
            nn.InstanceNorm2d(dim),
            nn.ReLU(True)]
        conv_block += [nn.ReflectionPad2d(1)]
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=0, bias=True),
            nn.InstanceNorm2d(dim)]
        return nn.Sequential(*conv_block)
    def forward(self, x):
        out = x + self.conv_block(x)
        return out
class ResNetGenerator(nn.Module):
    def __init__(self, input_nc=3, output_nc=3, ngf=64, n_blocks=9):
        assert(n_blocks >= 0)
        super(ResNetGenerator, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        self.ngf = ngf
        model = [nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=True),
            nn.InstanceNorm2d(ngf),
            nn.ReLU(True)]
        n_downsampling = 2
        for i in range(n_downsampling):
            mult = 2**i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3,
                                stride=2, padding=1, bias=True),
                        nn.InstanceNorm2d(ngf * mult * 2),
                        nn.ReLU(True)]
        mult = 2**n_downsampling
        for i in range(n_blocks):
            model += [ResNetBlock(ngf * mult)]
        for i in range(n_downsampling):
            mult = 2**(n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                            kernel_size=3, stride=2,
                                            padding=1, output_padding=1,
                                            bias=True),
                    nn.InstanceNorm2d(int(ngf * mult / 2)),
                    nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]
        self.model = nn.Sequential(*model)
    def forward(self, input):
        return self.model(input)

In [8]:
netG = ResNetGenerator()
netG

ResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): ResNetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(256, 

In [9]:
netG.eval()

ResNetGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): ResNetBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): Conv2d(256, 

In [10]:
from torchvision import datasets, transforms

In [11]:
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('../data', train=True, download=True,
transform=transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.1307,), (0.3081,))
])),
batch_size=64, shuffle=True)

0it [00:00, ?it/s]

9920512it [00:02, 3451979.22it/s]                                                                                      


Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 64433.34it/s]                                                                                          
0it [00:00, ?it/s]

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz


1654784it [00:01, 1165582.28it/s]                                                                                      
0it [00:00, ?it/s]

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz


8192it [00:00, 18559.97it/s]                                                                                           


Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [13]:
model = Net()

In [14]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [17]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        print('Current loss', float(loss))
        loss.backward()
        optimizer.step()

Current loss 2.319706439971924
Current loss 2.3495187759399414
Current loss 2.3279473781585693
Current loss 2.346665859222412
Current loss 2.3251614570617676
Current loss 2.3214004039764404
Current loss 2.2953128814697266
Current loss 2.3304338455200195
Current loss 2.316721200942993
Current loss 2.3185863494873047
Current loss 2.2872002124786377
Current loss 2.286309242248535
Current loss 2.332578420639038
Current loss 2.285099744796753
Current loss 2.330815553665161
Current loss 2.2867507934570312
Current loss 2.3419690132141113
Current loss 2.26973819732666
Current loss 2.313197135925293
Current loss 2.251391649246216
Current loss 2.2805919647216797
Current loss 2.268322467803955
Current loss 2.3052985668182373
Current loss 2.3296523094177246
Current loss 2.2645273208618164
Current loss 2.300597667694092
Current loss 2.2811620235443115
Current loss 2.2341370582580566
Current loss 2.315387010574341
Current loss 2.2395412921905518
Current loss 2.252814769744873
Current loss 2.29261279

Current loss 1.06911301612854
Current loss 0.8910613656044006
Current loss 0.9863068461418152
Current loss 0.7878329753875732
Current loss 0.9755778312683105
Current loss 1.0007671117782593
Current loss 1.075425386428833
Current loss 0.9047873616218567
Current loss 1.1160507202148438
Current loss 0.8151493072509766
Current loss 1.1137765645980835
Current loss 0.8780174851417542
Current loss 0.8599405288696289
Current loss 0.7714104652404785
Current loss 0.7652464509010315
Current loss 0.8017802834510803
Current loss 0.719468355178833
Current loss 0.9004698395729065
Current loss 1.0109617710113525
Current loss 0.6614565253257751
Current loss 0.9178500771522522
Current loss 0.9368187785148621
Current loss 0.955737829208374
Current loss 1.1141314506530762
Current loss 0.9515825510025024
Current loss 0.9728010296821594
Current loss 0.8564280867576599
Current loss 0.8102515339851379
Current loss 0.8062869906425476
Current loss 0.8548086285591125
Current loss 0.9963603019714355
Current loss 

Current loss 0.5914981961250305
Current loss 0.5686436891555786
Current loss 0.513906717300415
Current loss 0.6830117702484131
Current loss 0.619269073009491
Current loss 0.5914660692214966
Current loss 0.6336109638214111
Current loss 0.6033694744110107
Current loss 0.5417604446411133
Current loss 0.7384907007217407
Current loss 0.5126184225082397
Current loss 0.5911581516265869
Current loss 0.5525863170623779
Current loss 0.5658043622970581
Current loss 0.6807975769042969
Current loss 0.6533166170120239
Current loss 0.7159587740898132
Current loss 0.5229442119598389
Current loss 0.6854548454284668
Current loss 0.6139398813247681
Current loss 0.6416068077087402
Current loss 1.0643616914749146
Current loss 0.7341518998146057
Current loss 0.5752025842666626
Current loss 0.5770941972732544
Current loss 0.6104668378829956
Current loss 0.6507388353347778
Current loss 0.8575507402420044
Current loss 0.5142086744308472
Current loss 0.5835220217704773
Current loss 0.6206725239753723
Current lo

Current loss 0.35965657234191895
Current loss 0.8016718029975891
Current loss 0.4513915777206421
Current loss 0.6841618418693542
Current loss 0.5003060102462769
Current loss 0.5536168217658997
Current loss 0.3516164720058441
Current loss 0.4121231734752655
Current loss 0.538953423500061
Current loss 0.4773269593715668
Current loss 0.5617587566375732
Current loss 0.456698477268219
Current loss 0.4296778440475464
Current loss 0.3998858332633972
Current loss 0.39309850335121155
Current loss 0.38233205676078796
Current loss 0.5486226677894592
Current loss 0.5773781538009644
Current loss 0.5368506908416748
Current loss 0.38079771399497986
Current loss 0.49434608221054077
Current loss 0.994977593421936
Current loss 0.42887982726097107
Current loss 0.5873978734016418
Current loss 0.47689586877822876
Current loss 0.5531265139579773
Current loss 0.5327783823013306
Current loss 0.5850542783737183
Current loss 0.46927309036254883
Current loss 0.5475810766220093
Current loss 0.6229119896888733
Cur

Current loss 0.3680031895637512
Current loss 0.5433939695358276
Current loss 0.5124121904373169
Current loss 0.49515315890312195
Current loss 0.30735698342323303
Current loss 0.4649353325366974
Current loss 0.48799484968185425
Current loss 0.41548460721969604
Current loss 0.3433522582054138
Current loss 0.4096894860267639
Current loss 0.26021313667297363
Current loss 0.34947431087493896
Current loss 0.7111955881118774
Current loss 0.4807716906070709
Current loss 0.34388768672943115
Current loss 0.32090532779693604
Current loss 0.5610097050666809
Current loss 0.3017995357513428
Current loss 0.4799567759037018
Current loss 0.46892645955085754
Current loss 0.3706998825073242
Current loss 0.30223798751831055
Current loss 0.8535680174827576
Current loss 0.5199532508850098
Current loss 0.5362796187400818
Current loss 0.3301500082015991
Current loss 0.49654585123062134
Current loss 0.5870035886764526
Current loss 0.430715948343277
Current loss 0.46775496006011963
Current loss 0.43638077378273

...

Current loss 0.38196703791618347
Current loss 0.3862178325653076
Current loss 0.16163818538188934
Current loss 0.3378400504589081
Current loss 0.16056585311889648
Current loss 0.270041823387146
Current loss 0.32009589672088623
Current loss 0.411052942276001
Current loss 0.08669339865446091
Current loss 0.21916094422340393
Current loss 0.24566571414470673
Current loss 0.17777244746685028
Current loss 0.20724758505821228
Current loss 0.10880452394485474
Current loss 0.19027234613895416
Current loss 0.14963451027870178
Current loss 0.12272083759307861
Current loss 0.08160404115915298
Current loss 0.14089342951774597
Current loss 0.09433449804782867
Current loss 0.16860289871692657
Current loss 0.3276293873786926
Current loss 0.43996375799179077
Current loss 0.16945773363113403
Current loss 0.25358060002326965
Current loss 0.09347152709960938
Current loss 0.2408435344696045
Current loss 0.15711063146591187
Current loss 0.1598004698753357
Current loss 0.27718299627304077
Current loss 0.1716

Current loss 0.16082672774791718
Current loss 0.1456611454486847
Current loss 0.102839894592762
Current loss 0.14225122332572937
Current loss 0.0723843052983284
Current loss 0.05351507291197777
Current loss 0.23397070169448853
Current loss 0.11152099072933197
Current loss 0.16454409062862396
Current loss 0.08502644300460815
Current loss 0.21399131417274475
Current loss 0.44159916043281555
Current loss 0.05330660939216614
Current loss 0.2831844687461853
Current loss 0.235028475522995
Current loss 0.09561391919851303
Current loss 0.22880472242832184
Current loss 0.22038330137729645
Current loss 0.19015449285507202
Current loss 0.16537876427173615
Current loss 0.30686837434768677
Current loss 0.19261790812015533
Current loss 0.33553969860076904
Current loss 0.21412113308906555
Current loss 0.08559541404247284
Current loss 0.1326378583908081
Current loss 0.15672554075717926
Current loss 0.09394187480211258
Current loss 0.04724115878343582
Current loss 0.14120233058929443
Current loss 0.071

Current loss 0.6034057140350342
Current loss 0.13250988721847534
Current loss 0.05955468863248825
Current loss 0.18027924001216888
Current loss 0.2719423174858093
Current loss 0.1423894613981247
Current loss 0.07712512463331223
Current loss 0.26184090971946716
Current loss 0.21793971955776215
Current loss 0.4492463767528534
Current loss 0.19614055752754211
Current loss 0.22675180435180664
Current loss 0.37342849373817444
Current loss 0.20076100528240204
Current loss 0.18094301223754883
Current loss 0.18820720911026
Current loss 0.07375345379114151
Current loss 0.06815016269683838
Current loss 0.08021756261587143
Current loss 0.21344153583049774
Current loss 0.1818626970052719
Current loss 0.12743324041366577
Current loss 0.04307661950588226
Current loss 0.1868395060300827
Current loss 0.16192054748535156
Current loss 0.10140246897935867
Current loss 0.06433823704719543
Current loss 0.08922243863344193
Current loss 0.12739406526088715
Current loss 0.059941425919532776
Current loss 0.083

Current loss 0.1391536146402359
Current loss 0.0777159333229065
Current loss 0.1266300529241562
Current loss 0.09867996722459793
Current loss 0.06796246021986008
Current loss 0.4582335948944092
Current loss 0.1744287610054016
Current loss 0.12313773483037949
Current loss 0.1457778364419937
Current loss 0.11225356161594391
Current loss 0.1963203400373459
Current loss 0.23970995843410492
Current loss 0.08455552160739899
Current loss 0.18566268682479858
Current loss 0.13770608603954315
Current loss 0.2644805610179901
Current loss 0.25097182393074036
Current loss 0.05321672558784485
Current loss 0.10729364305734634
Current loss 0.18521451950073242
Current loss 0.11653290688991547
Current loss 0.10400784760713577
Current loss 0.11712902784347534
Current loss 0.14315228164196014
Current loss 0.04970790073275566
Current loss 0.12471125274896622
Current loss 0.07135526090860367
Current loss 0.15030938386917114
Current loss 0.18649998307228088
Current loss 0.30121034383773804
Current loss 0.105

Current loss 0.30394434928894043
Current loss 0.09930019825696945
Current loss 0.09227710962295532
Current loss 0.23980125784873962
Current loss 0.06948620826005936
Current loss 0.20264993607997894
Current loss 0.12937043607234955
Current loss 0.11345226317644119
Current loss 0.17332802712917328
Current loss 0.09305373579263687
Current loss 0.10692594945430756
Current loss 0.10220286250114441
Current loss 0.2105952501296997
Current loss 0.15480028092861176
Current loss 0.16591176390647888
Current loss 0.20653587579727173
Current loss 0.10344233363866806
Current loss 0.21750661730766296


In [18]:
torch.save(model.state_dict(), 'mnist.pth')
pretrained_model = Net()
pretrained_model.load_state_dict(torch.load('mnist.pth'))

print(pretrained_model)

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)
